In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from simplicial_kuramoto import SimplicialComplex
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *
from simplicial_kuramoto.plotting import *

In [ ]:
def get_delauney_holes_multi(n_points, centre_holes, radius, points=[]):

    if len(points) == 0:
        x = np.random.rand(n_points)
        y = np.random.rand(n_points)
        points = np.vstack([x, y]).T

    tri = Delaunay(points)

    edge_list = []

    idx_inside = np.empty([0], dtype=int)
    for i in range(centre_holes.shape[0]):
        idx_inside = np.hstack(
            [idx_inside, encloses([centre_holes[i]], points, radius)[1]]
        )

    for t in tri.simplices:

        if t[0] not in idx_inside and t[1] not in idx_inside:
            edge_list.append([t[0], t[1]])

        if t[1] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[1], t[2]])

        if t[0] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[0], t[2]])

    graph = nx.Graph()
    # graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)

    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    g = graph.subgraph(Gcc[0])

    return g, points


def encloses(centre, points, radius):
    inside_hole = cdist(centre, points, "euclidean") <= radius
    idx_inside = np.where(inside_hole)

    return idx_inside

In [ ]:
centre_holes = np.array([[0.5, 0.5]])
n_points = 10
radius = 0.2

graph, points = get_delauney_holes_multi(n_points, centre_holes, radius)
pos = dict(enumerate(points))
nx.draw(graph, pos, node_size=30)
plt.show()

np.random.seed(0)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
initial_phase = np.concatenate(
    (np.eye(len(graph.edges)), -np.eye(len(graph.edges))), axis=0
).dot(initial_phase)


t_max = 500
n_t = 100

complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)

In [ ]:
omega_0 = np.zeros(complex_delaunay.n_edges)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
edge_result = integrate_edge_kuramoto(
    complex_delaunay, initial_phase, t_max, n_t, omega_0=omega_0
)
plot_edge_kuramoto(edge_result)

In [ ]:
omega_0 = np.ones(complex_delaunay.n_edges)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
edge_result = integrate_edge_kuramoto(
    complex_delaunay, initial_phase, t_max, n_t, omega_0=omega_0
)
plot_edge_kuramoto(edge_result)

In [ ]:
omega_0 = np.ones(2 * complex_delaunay.n_edges)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
initial_phase = np.concatenate(
    (np.eye(len(graph.edges)), -np.eye(len(graph.edges))), axis=0
).dot(initial_phase)
alpha_1 = 0.0
alpha_2 = np.pi
edge_result = integrate_edge_kuramoto_frustrated(
    complex_delaunay,
    initial_phase,
    t_max,
    n_t,
    omega_0=omega_0,
    alpha_1=alpha_1,
    alpha_2=alpha_2,
)
plot_edge_kuramoto(edge_result)

In [ ]:
omega_0 = np.ones(2 * complex_delaunay.n_edges)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
initial_phase = np.concatenate(
    (np.eye(len(graph.edges)), -np.eye(len(graph.edges))), axis=0
).dot(initial_phase)
alpha_1 = 0.01
alpha_2 = np.pi
edge_result = integrate_edge_kuramoto_frustrated(
    complex_delaunay,
    initial_phase,
    t_max,
    n_t,
    omega_0=omega_0,
    alpha_1=alpha_1,
    alpha_2=alpha_2,
)
plot_edge_kuramoto(edge_result)

In [ ]:
omega_0 = np.zeros(2 * complex_delaunay.n_edges)
initial_phase = np.random.uniform(0, 2 * np.pi, len(graph.edges))
initial_phase = np.concatenate(
    (np.eye(len(graph.edges)), -np.eye(len(graph.edges))), axis=0
).dot(initial_phase)
alpha_1 = 1
alpha_2 = np.pi
edge_result = integrate_edge_kuramoto_frustrated(
    complex_delaunay,
    initial_phase,
    t_max,
    n_t,
    omega_0=omega_0,
    alpha_1=alpha_1,
    alpha_2=alpha_2,
)
plot_edge_kuramoto(edge_result)

In [ ]:
plt.figure()
nx.draw_networkx_nodes(graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    graph,
    pos=points,
    edge_color=edge_result.y[: graph.number_of_edges(), -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
)

In [ ]:
plt.figure()
nx.draw_networkx_nodes(graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    graph,
    pos=points,
    edge_color=edge_result.y[-graph.number_of_edges() :, -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
)